In [ ]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [ ]:
import logging
import delta_sharing
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [ ]:
customer = "Pendix"
fromdate = "20250806"
todate = "20250806"
source = "googleAdwords"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [ ]:
content={
     "fromDate":fromdate,
     "toDate":todate
}
audience_usage=send_to_innkeepr_api_paginated(
    f"{url}/audiences/usage/query",
    account_id,
    content,
    logging
)
audience_usage = pd.json_normalize(audience_usage)
audience_usage.columns

In [ ]:
au = audience_usage#[audience_usage["sourceName"] == source]

In [ ]:
au

In [ ]:
au[["date","spend","treatment","sourceName"]]

# Match Audiences with Treatments

In [ ]:
sources = call_api_with_accountId(
    f"{url}/sources/query",
    account_id,
    {"name":source},
    logging
)
if len(sources) != 1:
    raise Exception(f"Found {len(sources)} sources with name {source}")
source_id = sources[0]

In [ ]:
treatments =send_to_innkeepr_api_paginated(
    f"{url}/treatments/query",
    account_id,
    {"source":source_id["id"]},
    logging
)
treatments = pd.json_normalize(treatments)
treatments

In [ ]:
treatments_filtered = treatments[(treatments["relates_to.assetGroup.resourceName"].isin(au["treatment"]))|(treatments["relates_to.adGroup.resourceName"].isin(au["treatment"]))]
treatments_filtered = treatments_filtered[["name","relates_to.assetGroup.resourceName","relates_to.adGroup.resourceName","properties.status","relates_to.campaign.name","properties.type"]]
treatments_filtered

In [ ]:
print(len(au))
au = pd.merge(au, treatments_filtered, left_on="treatment", right_on="relates_to.assetGroup.resourceName", how="left")
print(len(au))
au

In [ ]:
au[au["date"]=="20250806"][["treatment","relates_to.assetGroup.resourceName","relates_to.campaign.name","name","spend","date"]].sort_values(by="relates_to.campaign.name")

In [ ]:
au["relates_to.campaign.name"].nunique()

In [ ]:
au[au["relates_to.campaign.name"].str.contains("Haar").fillna(False)]["relates_to.campaign.name"].drop_duplicates()